In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import zipfile
with zipfile.ZipFile('/content/drive/MyDrive/New_120dog breeds-299.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/projectDataset299')

In [ ]:

import zipfile
with zipfile.ZipFile('/content/drive/MyDrive/120dog breeds-299.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/projectDataset299')

In [ ]:
import pandas as pd
import pathlib
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras import applications
from keras.layers import Activation, Dropout, Flatten, Dense,GlobalAveragePooling2D, BatchNormalization

In [ ]:
import pathlib
from keras.preprocessing.image import ImageDataGenerator

data_dir = pathlib.Path('/content/projectDataset299/New_120dog breeds-299')

data_train = data_dir / 'train'

test_dir = data_dir / 'test'


labels_csv = pd.read_csv('/content/drive/MyDrive/new_data.csv')
labels_csv['id'] = [str(fname) + '.jpg' for fname in labels_csv['id']]

In [ ]:
batch_size = 16
img_size = 299

In [ ]:
# datagen = ImageDataGenerator(preprocessing_function = keras.applications.inception_v3.preprocess_input, validation_split=0.2)
# train_generator = datagen.flow_from_dataframe(
#                         dataframe = labels_csv,
#                         directory = data_train,
#                         subset='training',
#                         x_col='id',
#                         y_col='breed',
#                         batch_size=batch_size,
#                         shuffle=True,
#                         class_mode='sparse',
#                         color_mode='rgb',
#                         target_size=(img_size, img_size))

# validation_generator = datagen.flow_from_dataframe(
#                         dataframe = labels_csv,
#                         directory = data_train,
#                         subset='validation',
#                         x_col='id',
#                         y_col='breed',
#                         batch_size=batch_size,
#                         shuffle=True,
#                         class_mode='sparse',
#                         color_mode='rgb',
#                         target_size=(img_size, img_size))

# train_datagen = ImageDataGenerator(
#     preprocessing_function = keras.applications.inception_v3.preprocess_input, 
#     validation_split=0.2,
#     rotation_range=20, 
#     width_shift_range=0.2, 
#     height_shift_range=0.2, 
#     zoom_range=0.2, 
#     horizontal_flip=True, 
#     vertical_flip=True
# )

from keras.utils import to_categorical
train_datagen = ImageDataGenerator(
    preprocessing_function=keras.applications.inception_v3.preprocess_input,
    validation_split=0.2,
    rotation_range=5,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    shear_range=0.05,
    brightness_range=[0.8, 1.2],
    horizontal_flip=True
)


train_generator = train_datagen.flow_from_dataframe(
    dataframe=labels_csv,
    directory=data_train,
    subset='training',
    x_col='id',
    y_col='breed',
    batch_size=batch_size,
    shuffle=True,
    # class_mode='sparse',
    class_mode='categorical',
    color_mode='rgb',
    target_size=(img_size, img_size)
)

validation_datagen = ImageDataGenerator(
    preprocessing_function = keras.applications.inception_v3.preprocess_input, 
    validation_split=0.2
)

validation_generator = validation_datagen.flow_from_dataframe(
    dataframe=labels_csv,
    directory=data_train,
    subset='validation',
    x_col='id',
    y_col='breed',
    batch_size=batch_size,
    shuffle=True,
    # class_mode='sparse',
    class_mode='categorical',
    color_mode='rgb',
    target_size=(img_size, img_size)
)



train_labels = to_categorical(train_generator.labels)
validation_labels = to_categorical(validation_generator.labels)

Found 7218 validated image filenames belonging to 120 classes.
Found 1804 validated image filenames belonging to 120 classes.


In [ ]:
import tensorflow as tf
from tensorflow.python.keras.metrics import AUC
from sklearn.metrics import roc_curve, auc

# Define the ROC metrics
tpr = tf.keras.metrics.TruePositives(name='tp')
fpr = tf.keras.metrics.FalsePositives(name='fp')
auc_metric = AUC(name='auc')

In [ ]:
# base = applications.inception_v3.InceptionV3(include_top=False, weights='imagenet',input_shape=(224,224,3))


base = applications.inception_v3.InceptionV3(include_top=False, weights='imagenet', input_shape=(img_size, img_size, 3))


for layer in base.layers:
    layer.trainable=False

model = Sequential()
model.add(base)
model.add(BatchNormalization())
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(120, activation='softmax'))
model.summary()

optimizer=tf.keras.optimizers.Adam(learning_rate = 0.0001)
model.compile(optimizer=optimizer,
              # loss='sparse_categorical_crossentropy',
              loss='categorical_crossentropy',
              # metrics=['accuracy'])
              metrics=[tpr, fpr, auc_metric, 'accuracy', 'Precision', 'Recall'])

87910968/87910968 [==============================] - 4s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 8, 8, 2048)        21802784  
                                                                 
 batch_normalization_94 (Bat  (None, 8, 8, 2048)       8192      
 chNormalization)                                                
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 1024)              2098176   
                                                         

In [ ]:
# history_inc = model.fit(train_generator, epochs=50, verbose = 1,
#                         validation_data=validation_generator)

from keras.callbacks import ModelCheckpoint, EarlyStopping


checkpoint_cb = ModelCheckpoint("E://dx//Level 6//Project//Topic-Ensemble Model for the Classification of Dog Breed//Codes//Saved Model//InceptionV3-299-Epoch50New120BreedsDataset_HasDA+EarlyStopping.h5", save_best_only=True)
early_stopping_cb = EarlyStopping(patience=5, restore_best_weights=True)


history_inc = model.fit(train_generator, epochs=50, verbose = 1,
                        validation_data=validation_generator,
                        callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/50
452/452 [==============================] - 224s 460ms/step - loss: 2.5927 - tp: 1022.0000 - fp: 66.0000 - auc: 0.7887 - accuracy: 0.4950 - precision: 0.9393 - recall: 0.1416 - val_loss: 7.8284 - val_tp: 11.0000 - val_fp: 356.0000 - val_auc: 0.8573 - val_accuracy: 0.0078 - val_precision: 0.0300 - val_recall: 0.0061
Epoch 2/50
452/452 [==============================] - 201s 446ms/step - loss: 0.7413 - tp: 4760.0000 - fp: 359.0000 - auc: 0.8627 - accuracy: 0.8354 - precision: 0.9299 - recall: 0.6595 - val_loss: 9.5581 - val_tp: 14.0000 - val_fp: 820.0000 - val_auc: 0.8747 - val_accuracy: 0.0078 - val_precision: 0.0168 - val_recall: 0.0078
Epoch 3/50
452/452 [==============================] - 201s 444ms/step - loss: 0.5046 - tp: 5710.0000 - fp: 511.0000 - auc: 0.8774 - accuracy: 0.8599 - precision: 0.9179 - recall: 0.7911 - val_loss: 10.5190 - val_tp: 14.0000 - val_fp: 1036.0000 - val_auc: 0.8815 - val_accuracy: 0.0078 - val_precision: 0.0133 - val_recall: 0.0078
Epoch 4/50
452/

KeyboardInterrupt: ignored

In [ ]:
plt.figure()
plt.plot(history_inc .history['val_loss'], label = 'Val loss')
plt.plot(history_inc .history['loss'], label = 'Train loss')
plt.legend()
plt.show()
plt.figure()
plt.plot(history_inc .history['val_accuracy'], label = 'Val accuracy')
plt.plot(history_inc .history['accuracy'], label = 'Accuracy')
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.plot(history_inc .history['val_auc'], label = 'val_auc')
plt.plot(history_inc .history['auc'], label = 'Train auc')
plt.legend()
plt.show()

plt.figure()
plt.plot(history_inc .history['val_tp'], label = 'Val TP')
plt.plot(history_inc .history['tp'], label = 'Train TP')
plt.plot(history_inc .history['val_fp'], label = 'Val FP')
plt.plot(history_inc .history['fp'], label = 'Train FP')
plt.legend()
plt.show()

plt.figure()
plt.plot(history_inc .history['val_precision'], label = 'Val Precision')
plt.plot(history_inc .history['precision'], label = 'Train Precision')
plt.legend()
plt.show()

plt.figure()
plt.plot(history_inc .history['val_recall'], label = 'Val Recall')
plt.plot(history_inc .history['recall'], label = 'Train Recall')
plt.legend()
plt.show()



In [ ]:
model.save('/content/drive/MyDrive/InceptionV3-299-Epoch50DataAugmentationHighEarlyStopping.h5')